In [1]:
!pip install tensorflow==2.9.0
!pip install ampligraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 27.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 24.3.25
    Uninstalling flatbuffers-24.3.25:
      Successfully uninstalled flatbuffers-24.3.25
  Attempting uninstall: tensorboard-data-serve

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.3/210.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.5/575.5 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.0/567.0 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import ampligraph
import tensorflow as tf
from rdflib import Graph

In [3]:
g = Graph()
g.parse("civ6-ontology-v2.rdf", format="xml")

<Graph identifier=N3fde9692ad8c4e8ab8a8c7de08a2e1ec (<class 'rdflib.graph.Graph'>)>

In [7]:
triples = []
for s, p, o in g:
    triples.append((str(s), str(p), str(o)))

triples_df = pd.DataFrame(triples, columns=["subject", "predicate", "object"]).dropna()
triples_df.to_csv("triples.csv", index=False)
print(f"Extracted {len(triples)} triples.")

Extracted 2992 triples.


In [16]:
triples_filtered = triples_df[triples_df.object != 'None']

In [18]:
from ampligraph.latent_features import ScoringBasedEmbeddingModel
from ampligraph.evaluation import mrr_score, hits_at_n_score, train_test_split_no_unseen
from ampligraph.latent_features.loss_functions import get as get_loss
from ampligraph.latent_features.regularizers import get as get_regularizer
import tensorflow as tf

triples = triples_filtered.values

X_train, X_valid = train_test_split_no_unseen(triples, test_size=0.2)

model = ScoringBasedEmbeddingModel(k=100, eta=10, scoring_type="ComplEx")
optim = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss = get_loss("multiclass_nll")
regularizer = get_regularizer("LP", {"p": 2, "lambda": 1e-5})
model.compile(optimizer=optim, loss=loss, entity_relation_regularizer=regularizer)

model.fit(X_train)

Epoch 1/100
4/4 [==============================] - 2s 420ms/step - loss: 1905.5762
Epoch 2/100
4/4 [==============================] - 0s 62ms/step - loss: 1905.0490
Epoch 3/100
4/4 [==============================] - 0s 59ms/step - loss: 1904.5653
Epoch 4/100
4/4 [==============================] - 0s 60ms/step - loss: 1904.0776
Epoch 5/100
4/4 [==============================] - 0s 71ms/step - loss: 1903.5818
Epoch 6/100
4/4 [==============================] - 1s 147ms/step - loss: 1903.0736
Epoch 7/100
4/4 [==============================] - 1s 146ms/step - loss: 1902.5402
Epoch 8/100
4/4 [==============================] - 1s 159ms/step - loss: 1901.9771
Epoch 9/100
4/4 [==============================] - 1s 194ms/step - loss: 1901.3756
Epoch 10/100
4/4 [==============================] - 0s 86ms/step - loss: 1900.7308
Epoch 11/100
4/4 [==============================] - 0s 96ms/step - loss: 1900.0397
Epoch 12/100
4/4 [==============================] - 0s 90ms/step - loss: 1899.2896
Epoch 13

In [20]:
filter = {'test': X_valid, 'train': X_train}

ranks = model.evaluate(X_valid, use_filter=filter, corrupt_side="s,o")

mrr = mrr_score(ranks)
hits_10 = hits_at_n_score(ranks, n=10)
print("MRR: %f, Hits@10: %f" % (mrr, hits_10))

61/61 [==============================] - 6s 95ms/step
MRR: 0.553821, Hits@10: 0.653523


### Предсказание религии лидера на основе его принадлежности к определенной цивилизации


In [38]:
import numpy as np
from rdflib import Namespace
from sklearn.model_selection import train_test_split

ns = Namespace("http://n4d.sh/civ6#")

triples_rules = triples_df[triples_df['predicate'] == ns.rules.toPython()]
triples_beliefs = triples_df[triples_df['predicate'] == ns.believesIn.toPython()]

In [40]:
triples_rules.head(3)

,subject,predicate,object
49,http://n4d.sh/civ6#Abraham_Lincoln,http://n4d.sh/civ6#rules,http://n4d.sh/civ6#America
97,http://n4d.sh/civ6#Cleopatra_(Ptolemaic),http://n4d.sh/civ6#rules,http://n4d.sh/civ6#Egypt
104,"http://n4d.sh/civ6#Suleiman_I,_The_Magnificent",http://n4d.sh/civ6#rules,http://n4d.sh/civ6#Ottomans


In [41]:
triples_beliefs.head(3)

,subject,predicate,object
72,http://n4d.sh/civ6#Wilfrid_Laurier,http://n4d.sh/civ6#believesIn,http://n4d.sh/civ6#Catholicism
192,http://n4d.sh/civ6#Robert_the_Bruce,http://n4d.sh/civ6#believesIn,http://n4d.sh/civ6#Catholicism
234,http://n4d.sh/civ6#Dido_of_Carthage,http://n4d.sh/civ6#believesIn,http://n4d.sh/civ6#Judaism


In [42]:
train_beliefs, test_beliefs = train_test_split(triples_beliefs, test_size=0.2, random_state=42)
train_triples = pd.concat([triples_rules, train_beliefs])

print(f"Training triples: {len(train_triples)}")
print(f"Testing triples: {len(test_beliefs)}")

Training triples: 144
Testing triples: 11


In [45]:
train_triples_array = train_triples.values
test_triples_array = test_beliefs.values

model.fit(train_triples_array)

ranks = model.evaluate(test_triples_array, use_filter={'test': train_triples_array}, corrupt_side="s,o")

mrr = mrr_score(ranks)
hits_10 = hits_at_n_score(ranks, n=10)
print(f"MRR: {mrr:.4f}, Hits@10: {hits_10:.4f}")


4 triples containing invalid keys skipped! You can use `ScoringBasedEmbeddingModel.get_invalid_keys` or`DataIndexer.get_invalid_keys` to find out which keys are invalid. 

Epoch 1/100
2/2 [==============================] - 0s 62ms/step - loss: 1049.4889
Epoch 2/100
2/2 [==============================] - 0s 43ms/step - loss: 1046.2091
Epoch 3/100
2/2 [==============================] - 0s 46ms/step - loss: 1042.9523
Epoch 4/100
2/2 [==============================] - 0s 47ms/step - loss: 1039.7162
Epoch 5/100
2/2 [==============================] - 0s 41ms/step - loss: 1036.4869
Epoch 6/100
2/2 [==============================] - 0s 47ms/step - loss: 1033.2836
Epoch 7/100
2/2 [==============================] - 0s 29ms/step - loss: 1030.1017
Epoch 8/100
2/2 [==============================] - 0s 28ms/step - loss: 1026.9274
Epoch 9/100
2/2 [==============================] - 0s 34ms/step - loss: 1023.7637
Epoch 10/100
2/2 [==============================] - 0s 27ms/step - loss: 1020.6166
Epoch 

In [73]:
leader = ns.Peter_I
religions = triples_beliefs.object.unique()

candidates = [[leader.toPython(), ns.believesIn.toPython(), religion] for religion in religions]

scores = model.predict(candidates)
scores

array([5.9018364 , 0.97977626, 2.6452236 , 7.91105   , 5.4735284 ,
       4.5192437 , 5.4928856 , 1.8328092 , 4.2020807 , 2.962931  ],
      dtype=float32)

In [74]:
import numpy as np
best_religion = candidates[np.argmax(scores)][2]

print(f"Predicted Religion for {leader}: {best_religion}")

Predicted Religion for http://n4d.sh/civ6#Peter_I: http://n4d.sh/civ6#Eastern_Orthodoxy
